In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# تعریف مدل MLP
class MLPModel(nn.Module):
    def __init__(self):
        super(MLPModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 512)  # لایه مخفی اول
        self.fc2 = nn.Linear(512, 256)    # لایه مخفی دوم
        self.fc3 = nn.Linear(256, 128)    # لایه مخفی سوم
        self.fc4 = nn.Linear(128, 10)     # لایه خروجی (۱۰ کلاس برای MNIST)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return self.softmax(x)

# مقداردهی اولیه مدل، تابع خطا و بهینه‌ساز
model = MLPModel()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# تنظیمات پیش‌پردازش داده‌ها (برای نرمال‌سازی تصاویر)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # نرمال‌سازی تصاویر
])

# بارگذاری دیتاست MNIST (آموزش و تست)
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# بارگذاری داده‌ها
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# آموزش مدل
def train_model(model, trainloader, optimizer, loss_fn, epochs=5):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for data in trainloader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(trainloader)}, Accuracy: {100 * correct / total}%")

# آموزش مدل بدون Augmentation
train_model(model, trainloader, optimizer, loss_fn)


Epoch 1/5, Loss: 1.650960226684237, Accuracy: 81.27166666666666%
Epoch 2/5, Loss: 1.5428694349362144, Accuracy: 91.89%
Epoch 3/5, Loss: 1.5326990269140395, Accuracy: 92.835%
Epoch 4/5, Loss: 1.5250562452304084, Accuracy: 93.59666666666666%
Epoch 5/5, Loss: 1.521567966256823, Accuracy: 93.95166666666667%
